In [23]:
%matplotlib inline
import sqlite3
import pandas as pd
import os
import numpy as np
import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func
from flask import jsonify

In [22]:
# !pip install sqlalchemy
# !pip install flask

In [2]:
# List of folders containing the CSV files
folders = ['./Resources/arefin_data', './Resources/ellis_data', './Resources/rita_data', './Resources/uwagboe_data']

# Connect to SQLite database (this will create the file if it doesn't exist)
conn = sqlite3.connect('stockdata.sqlite')

for folder in folders:
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            # Read CSV file into DataFrame
            df = pd.read_csv(os.path.join(folder, file))
            
            # Check if 'id' column exists, if not create it
            if 'id' not in df.columns:
                df.insert(0, 'id', range(1, 1 + len(df)))
            
            # Use the filename (without .csv) as the table name
            table_name = os.path.splitext(file)[0]
            
            # Convert DataFrame to SQL table in SQLite database
            df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [3]:
# Now we retrieve the data from the database
# engine = create_engine("sqlite:///stockdata.sqlite")

In [4]:
# # reflect an existing database into a new model
# Base = automap_base() # Declare a Base using `automap_base()`
# # reflect the tables
# Base.prepare(engine, reflect=True) # Use the Base class to reflect the database tables

In [5]:
# # View all of the classes that automap found
# print(Base.classes.keys()) # Print all of the classes mapped to the Base

In [15]:
from sqlalchemy import inspect

engine = create_engine("sqlite:///stockdata.sqlite")
# Use the inspector to get the table names
inspector = inspect(engine)
table_names = inspector.get_table_names()

print(table_names)

# conn = engine.connect()
# result = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = result.fetchall()
# print(tables)
# conn.close()

['AAPL', 'AMZN', 'ANSS', 'DXC', 'FFIV', 'GE', 'GOOGL', 'HPE', 'JNPR', 'KEYS', 'META', 'MSFT', 'MTD', 'NDVA', 'QRVO', 'S&P500companies', 'S&P500companies_technology', 'SEDG', 'TSLA', 'ZBH']


In [16]:
# Query data into a DataFrame by starting with the main comprehensive list 'S&P500companies'
conn = sqlite3.connect("stockdata.sqlite")
df = pd.read_sql_query("SELECT * FROM [S&P500companies]", conn)
df.head()

,id,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,LogoPath
0,1,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,NotAvailable
1,2,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,NotAvailable
2,3,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,NotAvailable
3,4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),NotAvailable
4,5,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,NotAvailable


In [25]:
print(df.head())

   id Ticker      Company                      Sub_sector  \
0   1    MMM           3M        Industrial Conglomerates   
1   2    AOS  A. O. Smith               Building Products   
2   3    ABT       Abbott           Health Care Equipment   
3   4   ABBV       AbbVie                 Pharmaceuticals   
4   5    ACN    Accenture  IT Consulting & Other Services   

              Headquarters  Date added      Founded      LogoPath  
0    Saint Paul, Minnesota  1957-03-04         1902  NotAvailable  
1     Milwaukee, Wisconsin  2017-07-26         1916  NotAvailable  
2  North Chicago, Illinois  1957-03-04         1888  NotAvailable  
3  North Chicago, Illinois  2012-12-31  2013 (1888)  NotAvailable  
4          Dublin, Ireland  2011-07-06         1989  NotAvailable  


In [17]:
# Rename some columns
df = df.rename(columns={'Symbol':'Ticker','Security':'Company','GICS Sub-Industry': 'Sub_sector', 'Headquarters Location': 'Headquarters'})
df.head()

,id,Ticker,Company,GICS Sector,Sub_sector,Headquarters,Date added,CIK,Founded,LogoPath
0,1,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,NotAvailable
1,2,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,NotAvailable
2,3,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,NotAvailable
3,4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),NotAvailable
4,5,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,NotAvailable


In [18]:
# Delete some columns 
df = df.drop(columns=['GICS Sector', 'CIK'])
# Show what's the current state of the df
df.head()

,id,Ticker,Company,Sub_sector,Headquarters,Date added,Founded,LogoPath
0,1,MMM,3M,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,1902,NotAvailable
1,2,AOS,A. O. Smith,Building Products,"Milwaukee, Wisconsin",2017-07-26,1916,NotAvailable
2,3,ABT,Abbott,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1888,NotAvailable
3,4,ABBV,AbbVie,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,2013 (1888),NotAvailable
4,5,ACN,Accenture,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1989,NotAvailable


In [19]:
# Construct the dictionaries
metadata = df.to_dict(orient='records')
# Print the resulting dictionary to verify
# print(data_dict)
metadata

[{'id': 1,
  'Ticker': 'MMM',
  'Company': '3M',
  'Sub_sector': 'Industrial Conglomerates',
  'Headquarters': 'Saint Paul, Minnesota',
  'Date added': '1957-03-04',
  'Founded': '1902',
  'LogoPath': 'NotAvailable'},
 {'id': 2,
  'Ticker': 'AOS',
  'Company': 'A. O. Smith',
  'Sub_sector': 'Building Products',
  'Headquarters': 'Milwaukee, Wisconsin',
  'Date added': '2017-07-26',
  'Founded': '1916',
  'LogoPath': 'NotAvailable'},
 {'id': 3,
  'Ticker': 'ABT',
  'Company': 'Abbott',
  'Sub_sector': 'Health Care Equipment',
  'Headquarters': 'North Chicago, Illinois',
  'Date added': '1957-03-04',
  'Founded': '1888',
  'LogoPath': 'NotAvailable'},
 {'id': 4,
  'Ticker': 'ABBV',
  'Company': 'AbbVie',
  'Sub_sector': 'Pharmaceuticals',
  'Headquarters': 'North Chicago, Illinois',
  'Date added': '2012-12-31',
  'Founded': '2013 (1888)',
  'LogoPath': 'NotAvailable'},
 {'id': 5,
  'Ticker': 'ACN',
  'Company': 'Accenture',
  'Sub_sector': 'IT Consulting & Other Services',
  'Headquarte

In [26]:
# metadata2 = jsonify(df)
# metadata2

In [28]:
df = pd.read_sql_query("SELECT * FROM AAPL", conn)
df

,id,Date,Open,High,Low,Close,Adj Close,Volume
0,1,2018-01-01,42.540001,45.025002,41.174999,41.857498,39.633190,2638717600
1,2,2018-02-01,41.792500,45.154999,37.560001,44.529999,42.163677,3711577200
2,3,2018-03-01,44.634998,45.875000,41.235001,41.945000,39.877968,2854910800
3,4,2018-04-01,41.660000,44.735001,40.157501,41.314999,39.279011,2664617200
4,5,2018-05-01,41.602501,47.592499,41.317501,46.717499,44.415291,2483905200
...,...,...,...,...,...,...,...,...
62,63,2023-03-01,146.830002,165.000000,143.899994,164.899994,164.450150,1520266600
63,64,2023-04-01,164.270004,169.850006,159.779999,169.679993,169.217117,969709700
64,65,2023-05-01,169.279999,179.350006,164.309998,177.250000,176.766464,1275155500
65,66,2023-06-01,177.699997,194.479996,176.929993,193.970001,193.708420,1297101100


In [31]:
#5 Acquire the Records
set0 = pd.read_sql_query("SELECT * FROM AAPL", conn) #pd.read_csv('./Resources/ellis_data/AAPL.csv')
set1 = pd.read_sql_query("SELECT * FROM AMZN", conn) #pd.read_csv('./Resources/ellis_data/AMZN.csv')
set2 = pd.read_sql_query("SELECT * FROM GOOGL", conn) #pd.read_csv('./Resources/ellis_data/GOOGL.csv')
set3 = pd.read_sql_query("SELECT * FROM META", conn) #pd.read_csv('./Resources/ellis_data/META.csv')
set4 = pd.read_sql_query("SELECT * FROM MSFT", conn) #pd.read_csv('./Resources/ellis_data/MSFT.csv')
set5 = pd.read_sql_query("SELECT * FROM NDVA", conn) #pd.read_csv('./Resources/ellis_data/NDVA.csv')
set6 = pd.read_sql_query("SELECT * FROM TSLA", conn) #pd.read_csv('./Resources/ellis_data/TSLA.csv')
#------------
set7 = pd.read_sql_query("SELECT * FROM DXC", conn) #pd.read_csv('./Resources/uwagboe_data/DXC.csv')
set8 = pd.read_sql_query("SELECT * FROM FFIV", conn) #pd.read_csv('./Resources/uwagboe_data/FFIV.csv')
set9 = pd.read_sql_query("SELECT * FROM JNPR", conn) #pd.read_csv('./Resources/uwagboe_data/JNPR.csv')
set10 = pd.read_sql_query("SELECT * FROM QRVO", conn) #pd.read_csv('./Resources/uwagboe_data/QRVO.csv')
set11 = pd.read_sql_query("SELECT * FROM SEDG", conn) #pd.read_csv('./Resources/uwagboe_data/SEDG.csv')
#------------
set12 = pd.read_sql_query("SELECT * FROM ANSS", conn) #pd.read_csv('./Resources/arefin_data/ANSS-Ansys_monthly_5yrs.csv')
set13 = pd.read_sql_query("SELECT * FROM GE", conn) #pd.read_csv('./Resources/arefin_data/GE-General Electric_monthly_5yrs.csv')
set14 = pd.read_sql_query("SELECT * FROM HPE", conn) #pd.read_csv('./Resources/arefin_data/HPE-Hawlett Packard_monthly_5yrs.csv')
set15 = pd.read_sql_query("SELECT * FROM KEYS", conn) #pd.read_csv('./Resources/arefin_data/KEYS-Keysight Holdings_monthly_5yrs.csv')
set16 = pd.read_sql_query("SELECT * FROM MTD", conn) #pd.read_csv('./Resources/arefin_data/MTD-Mettler Toledo_monthly_5yrs.csv')
set17 = pd.read_sql_query("SELECT * FROM ZBH", conn) #pd.read_csv('./Resources/arefin_data/ZBH-Zimmer Biomet Holdings_monthly_5yrs.csv')

df_sets = ["set" + str(i) for i in range(18)]

tickers = ["AAPL","AMZN","GOOGL","META","MSFT","NVDA","TSLA","DXC","FFIV","JNPR","QRVO","SEDG","ANSS","GE","HPE","KEYS","MTD","ZBH"] # df["Ticker"].tolist()



In [32]:
df_sets

['set0',
 'set1',
 'set2',
 'set3',
 'set4',
 'set5',
 'set6',
 'set7',
 'set8',
 'set9',
 'set10',
 'set11',
 'set12',
 'set13',
 'set14',
 'set15',
 'set16',
 'set17']

In [30]:
(globals()[df_sets[0]])["Date"]

0     2018-01-01
1     2018-02-01
2     2018-03-01
3     2018-04-01
4     2018-05-01
         ...    
62    2023-03-01
63    2023-04-01
64    2023-05-01
65    2023-06-01
66    2023-07-01
Name: Date, Length: 67, dtype: object

In [ ]:
globals()[df_sets[0]]

In [ ]:
stock_records = []
for cdf, ts in zip(df_sets, tickers):
    company_record={
        "ticker":ts,
        "dates":(globals()[cdf])["Date"].tolist(),
        "open": (globals()[cdf])["Open"].tolist(),
        "high": (globals()[cdf])["High"].tolist(),
        "low": (globals()[cdf])["Low"].tolist(),
        "close": (globals()[cdf])["Close"].tolist(),
        "adj close": (globals()[cdf])["Adj Close"].tolist(),
        "volume": (globals()[cdf])["Volume"].tolist()
    }
    stock_records.append(company_record)

final_dataset = {"tickers":tickers,
                 "company_info":metadata,
                 "stock_history": stock_records,
                }

print(final_dataset)

In [ ]:
import json

final_dataset = {
    "tickers": tickers,
    "company_info": metadata,
    "stock_history": stock_records
}

# Save the final_dataset as a JSON file
with open('final_dataset.json', 'w') as json_file:
    json.dump(final_dataset, json_file, indent=4)